In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

%load_ext autoreload
%autoreload 2
%matplotlib ipympl

from ndgsp.graph.graphs import Graph, ProductGraph

2023-02-22 15:45:12.961154: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Using Jax backend with device TFRT_CPU_0


In [3]:
def get_A_from_rules(A_A: np.ndarray, 
                     A_B: np.ndarray, 
                     A_C: np.ndarray, 
                     rules: list[int]):
    """
    Create a Cartesian product graph adjacency matrix using the factor 
    adjacency matrices using the product graph construction rules
    """
    
    A = len(A_A)
    B = len(A_B)
    C = len(A_C)
    
    out = np.zeros((A * B * C, A * B * C))

    nodes = np.argwhere(np.ones((A, B, C)))
    
    for i, (a, b, c) in enumerate(nodes): 
        
        for j, (a_, b_, c_) in enumerate(nodes):
            
            out[i, j] = out[j, i] = (A_A[a, a_] and b == b_ and c == c_) or (a == a_ and A_B[b, b_] and c == c_) or (a == a_ and b == b_ and A_C[c, c_])
            
    return out

In [4]:
from pykronecker import KroneckerSum

In [5]:
GA = Graph.random_connected(5)
GB = Graph.random_connected(6)
GC = Graph.random_connected(7)

np.allclose(KroneckerSum([GA.A, GB.A, GC.A]).to_array(), get_A_from_rules(GA.A, GB.A, GC.A, [1, 7]))

True

In [14]:
def create_3d_CPG_tex():

    def node (i: int, offset: tuple, colors: tuple):

        return f"""\\node ({i}) at ({offset[0]}, {offset[1]}) {{}};
    \\draw[gray, very thin, fill={{rgb,255: red,{colors[0][0]}; green,{colors[0][1]}; blue,{colors[0][2]}}}, shift={{({offset[0]}, {offset[1]})}}] (0,0) -- (0,0.5) arc[start angle=90, end angle=210, radius=0.5cm] -- (0,0);
    \\draw[gray, very thin, fill={{rgb,255: red,{colors[1][0]}; green,{colors[1][1]}; blue,{colors[1][2]}}}, shift={{({offset[0]}, {offset[1]})}}] (0,0) -- (-0.433,-0.25) arc[start angle=210, end angle=330, radius=0.5cm] -- (0,0);
    \\draw[gray, very thin, fill={{rgb,255: red,{colors[2][0]}; green,{colors[2][1]}; blue,{colors[2][2]}}}, shift={{({offset[0]}, {offset[1]})}}] (0,0) -- (0.433,-0.25) arc[start angle=-30, end angle=90, radius=0.5cm] -- (0,0);

    """

    cA = np.array([(255, 0, 40), (255, 100, 0), (255, 145, 0), (255, 205, 0), (255, 255, 0)])
    cB = np.array([(0, 255, 0), (0, 255, 139), (0, 255, 255), (0, 255, 200)])
    cC = np.array([(0, 0, 255), (140, 0, 255), (200, 0, 255), (255, 0, 255)])

    pA = np.array([(-5.5, 5.75), (-5.5, 2.5), (-4.5, -0.25), (-6.0, -1.75), (-4.75, -4.75)])
    pB = np.array([(-1.25, 2.0), (2.25, 1.75), (7.75, 2.25), (5.0, 0.25)])
    pC = np.array([(11, 1), (14, 4), (17, 1), (14, -2)]).astype(float)

    pA -= np.array(pA).mean(0)
    pB -= np.array(pB).mean(0)
    pC -= np.array(pC).mean(0)

    A_A = np.array([[0, 1, 0, 0, 0], 
                    [1, 0, 1, 1, 0], 
                    [0, 1, 0, 0, 1], 
                    [0, 1, 0, 0, 1], 
                    [0, 0, 1, 1, 0]]).astype(float)

    A_B = np.array([[0, 1, 0, 0], 
                    [1, 0, 1, 1], 
                    [0, 1, 0, 0], 
                    [0, 1, 0, 0]]).astype(float)

    A_C = np.array([[0, 1, 0, 1], 
                    [1, 0, 1, 0], 
                    [0, 1, 0, 1], 
                    [1, 0, 1, 0]]).astype(float)

    A = KroneckerSum([A_A, A_B, A_C]).to_array()


    out = r"""\documentclass[crop,tikz]{standalone}

    \usepackage{tikz}
    \usetikzlibrary{positioning, shapes.geometric, backgrounds}

    \begin{document}

    \begin{tikzpicture}

    """

    for i, (a, b, c) in enumerate(np.argwhere(np.ones((5, 4, 4)))):

        node_tex = node(i, offset=(pA[a] + pB[b] + 4 * pC[c]).round(3), colors=(cA[a], cB[b], cC[c]))

        out += node_tex

    out += r'\begin{scope}[on background layer]'
    out += '\n'

    for i, (a, b, c) in enumerate(np.argwhere(np.ones((5, 4, 4)))):

        for j, (a_, b_, c_) in enumerate(np.argwhere(np.ones((5, 4, 4)))):

            if j < i:
                continue

            if A[i, j]:

                out += f'\draw[gray, very thin] ({i}) to ({j});\n'

    out += r"""

    \end{scope}

    \end{tikzpicture}


    \end{document} """


    with open('diagram.tex', 'w') as f:

        f.write(out)


In [15]:
create_3d_CPG_tex()